In [2]:
import os
from collections import defaultdict
from glob import glob

import PIL
import numpy as np
import streamlit as st
import torch
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, CenterCrop
import torchvision.transforms.functional as F
from pymatting import *

from lib import U2NET_full
from lib.utils.oom import free_up_memory
import cv2

In [3]:
def load_samples(folder_path='./dataset/demo'):
    assert os.path.isdir(folder_path), f'Unable to open {folder_path}'
    samples = glob(os.path.join(folder_path, f'*.jpg'))
    print(os.getcwd())
    return samples

In [4]:
device = 'cuda'
samples = load_samples()
# model_select, sample_select = create_ui(samples)
model_select = 'checkpoint.pth'

E:\data\workspace\python\wanxiang-ai\u-2-net-portrait


In [5]:
def square_pad(image, fill=255):
    w, h = image.size
    max_wh = np.max([w, h])
    hp = int((max_wh - w) / 2)
    vp = int((max_wh - h) / 2)
    padding = (hp, vp, hp, vp)
    return F.pad(image, padding, fill, 'constant')


def get_transform():
    transforms = []
    # transforms.append(Resize(440)) # TBD: keep aspect ratio
    transforms.append(ToTensor())
    transforms.append(Normalize(mean=[.5, .5, .5],
                                std=[.5, .5, .5]))

    return Compose(transforms)

In [6]:
## 打印图片作为测试
def cv2plt(img):
    cv2.imshow("img", cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # print(new_image.astype(np.float32))
    #  add below code
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [13]:
device = 'cpu'
checkpoint = torch.load(f'./checkpoints/{model_select}', map_location=device)
model = U2NET_full().to(device=device)

if 'model' in checkpoint:
    model.load_state_dict(checkpoint['model'])
else:
    model.load_state_dict(checkpoint)

# 遍历每一张图片
# for sample_select in samples:
sample_select = samples[0]
image = Image.open(sample_select).convert('RGB')

#
transforms = get_transform()
alpha_image = None
model.eval()
with torch.no_grad():
    x = transforms(image)
    x = x.to(device=device).unsqueeze(dim=0)
    y_hat, _ = model(x)

    alpha_image = y_hat.mul(255)
    alpha_image = Image.fromarray(alpha_image.squeeze().cpu().detach().numpy()).convert('L')
    # st.image(alpha_image, width=800)

In [14]:
image = np.asarray(image)
# ##Option1: 绿背景
# background = np.zeros(image.shape)
# background[:, :] = [0, 177 / 255, 64 / 255]

# ##Option2: 透明背景
h, w, c = image.shape
background = np.zeros((h, w, 3))

alpha = y_hat.squeeze().cpu().detach()
alpha = np.asarray(alpha)
# alpha = (alpha * 255).astype(np.uint8)
image = image.astype(np.float32) / 255

foreground = estimate_foreground_ml(
    image, alpha)  # , n_big_iterations=1, n_small_iterations=1, regularization=10e-10

new_image = blend(foreground, background, alpha).astype(np.float32)
cv2plt(new_image)
## Combine the new_image (foreground + transparent background) with the alpha channel
alpha = alpha[..., np.newaxis]
rgba_image = np.concatenate((new_image, alpha), axis=2)
rgba_image = (rgba_image * 255).astype(np.uint8)
rgba_pil_image = Image.fromarray(rgba_image, mode="RGBA")

## Save the result as a PNG file
rgba_pil_image.save("res.png")

del y_hat
free_up_memory()

In [9]:
# image = np.asarray(image)
# alpha = y_hat.squeeze().cpu().detach()
# alpha = np.asarray(alpha)
# image = image.astype(np.float32) / 255
#
# foreground = estimate_foreground_ml(image, alpha)
#
# h, w, c = image.shape
# transparent_background = np.zeros((h, w, 3))
#
# new_image = blend(foreground, transparent_background, alpha)
#
# # Combine the new_image (foreground + transparent background) with the alpha channel
# alpha = alpha[..., np.newaxis]
# rgba_image = np.concatenate((new_image, alpha), axis=2)
# rgba_image = (rgba_image * 255).astype(np.uint8)
# rgba_pil_image = Image.fromarray(rgba_image, mode="RGBA")
#
# # Save the result as a PNG file
# rgba_pil_image.save("res.png")
#
# del y_hat
# free_up_memory()


In [10]:
# 抠图（原图，mask图，输出前景图）
image = np.asarray(image)

# alpha = y_hat.squeeze().cpu().detach()
# alpha = np.asarray(alpha)
# alpha = Image.fromarray(((alpha * 255).astype('uint8')))
# alpha.show()

alpha = y_hat.squeeze().cpu().detach()
alpha = np.asarray(alpha)
# alpha = (alpha * 255).astype(np.uint8)
image = image.astype(np.float32) / 255

foreground = estimate_foreground_ml(
    image, alpha)  # , n_big_iterations=1, n_small_iterations=1, regularization=10e-10

# print(type(foreground))
# cv2plt(foreground)

# h, w, c = image.shape
# img3 = np.zeros((h, w, 4))
# img3[:, :, 0:3] = image
# img3[:, :, 3] = alpha


alpha = alpha[..., np.newaxis]
rgba_image = np.concatenate((foreground, alpha), axis=2)
rgba_image = (rgba_image * 255).astype(np.uint8)
rgba_pil_image = Image.fromarray(rgba_image, mode="RGBA")
rgba_pil_image.save("res.png")


# img3 = Image.fromarray(img3, mode='RGBA')
# cv2.imwrite("res.png", img3)


NameError: name 'y_hat' is not defined

In [ ]:
# cv2plt(image)

In [ ]:
print(image.shape)
print(alpha.shape)